See eventual readme and explanation page for full comments and motivation on this project.

#### import statements

In [11]:
from trueskill import Rating, quality_1vs1, rate_1vs1, rate, TrueSkill
import trueskill

import pandas as pd
import numpy as np

import numpy.linalg as la
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import json
import requests
import re
import os
import itertools
import bs4
import networkx
import urllib.request
from itertools import combinations
from bs4 import BeautifulSoup
import collections
from collections import Counter

import time

import warnings
warnings.filterwarnings(action='ignore')

from functools import reduce

from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [12]:
#setting today's date for export statements

today = date.today()
today_dash = str(today.strftime("%Y")) + '-' + str(today.strftime("%m")) + '-' + str(today.strftime("%d"))
print(today_dash)

today = date.today()
today = str(today.strftime("%Y")) + str(today.strftime("%m")) + str(today.strftime("%d"))
print(today)

2021-09-27
20210927


#### custom functions for parsing scraped strings

In [13]:
times = []

for i in range(0, 60):
    '''converts listed matchtimes into separate variable in case i want to do something with that one day'''
    for j in range(0,60):
        if j < 10:
            times.append("("+str(i)+":0"+str(j)+")")
        else:
            times.append("("+str(i)+":"+str(j)+")")

### All below functions split matches into winners and losers depending on the match structure           
            
def singles_splitter(item):
    '''
    
    '''
    check = re.split('defeats', item)
    check = [item.rstrip().lstrip() for item in check]
    for item in check:
        if any(tme in item for tme in times):
            match_len = item[-7:]
            fixed = item[:-7]
            del check[-1]
            check.append(fixed)
            check.append(match_len.rstrip().lstrip())
            break
    return check

def multiway_splitter(item):
    check = re.split(' and ', item)
    check = [item.rstrip().lstrip() for item in check]
    for item in check:
        if any(tme in item for tme in times):
            match_len = item[-7:]
            fixed = item[:-7]
            del check[-1]
            check.append(fixed)
            check.append(match_len.rstrip().lstrip())
            break
    return check

def tag_splitter(item):
    check = re.split('defeat', item)
    check = [item.rstrip().lstrip() for item in check]
    for item in check:
        if any(tme in item for tme in times):
            match_len = item[-7:]
            fixed = item[:-7]
            del check[-1]
            check.append(fixed)
            check.append(match_len.rstrip().lstrip())
            break
    return check

def tag_team_parser(i):
    if re.search("\(", i):
        i = i.replace('(', ')')
        i = i.split(')')
        return(i[1])
    else:
        return(i)
    
def notes_stripper(item):
    '''removes misc bugs and issues'''
    item = item.replace(r" by referee's decision","")
    item = item.replace(r" by referee's decision","")
    item = item.replace(r" by Count Out","")
    item = item.replace(r" by Count Out","")
    item = item.replace(r" [2:1] - Kenny Omega pins PAC [2:1]","")
    return item

#### scraping cagematch for AEW events

In [14]:
webpage = 'https://www.cagematch.net/?id=8&nr=2287&page=4&s='
pages = ['0', '100', '200', '300']

all_events = []

for page in pages:
    cagematch_html = requests.get(str(webpage + page), headers={'Accept-Encoding': 'identity'})
    cagematch_html.encoding = cagematch_html.apparent_encoding
    soup = BeautifulSoup(cagematch_html.text, features='html.parser')
    links = []

    for link in soup.find_all('a'):
        links.append(link.get('href'))
        
    links2 = [item for item in links if item and '?id=1&' in item]
    links2 = [item for item in links2 if item and '&page=2' not in item]
    links2 = [item for item in links2 if item and 'view' not in item]
    for item in links2:
        all_events.append(item)

In [15]:
#test

webpage = 'https://www.cagematch.net/?id=1&nr=283430'

cagematch_html = requests.get(webpage, headers={'Accept-Encoding': 'identity'})
cagematch_html.encoding = cagematch_html.apparent_encoding
soup = BeautifulSoup(cagematch_html.text, features='html.parser')

soup.get_text()

'\n\n\n\n\n\n\n\n\n\n\nAEW Dynamite #45 « Events Database « CAGEMATCH - The Internet Wrestling Database\n\nwindow.onload = function() { loadComments("commentBox", "1", "283430", "", "en"); }\n\n\n\nNot logged in or registered. | Log In | Register | Password lost?\n\n\n\n\n\nAbout UsContactCAGEMATCH TeamYouTubeTwitterPatreon\n\nWrestlersOverviewWrestlersGimmicksTop ListsCommentsChangesEventsOverviewResultsTV + PPVUpcoming ShowsTop ListsSearch For EventsCommentsChangesMatchesOverviewMatchguideTop ListsSearch For EventsStatisticsCommentsChangesPromotionsOverviewPromotionsPromotion NamesTop ListsCommentsChangesTitlesOverviewTitle NamesTitle ChangesTop ListsCommentsChangesDatabasesTag TeamsStablesPromosFeudsRivalriesTournamentsAwardsBooksDVDsWrestling GamesMoviesThemesOtherSOTWArticlesPodcastsDocumentariesNewsboardChanges TrackerRandom ItemRatings CenterRatings System RulesHelp\n\n\n\n\n\n\nCAGEMATCH » \nEvents Database » \nAEW Dynamite #45\n\n\nAEW Dynamite #45TV-ShowResultsCardCard with g

#### Initial parse of results

In [106]:
event_titles = []
match_list = []

event_card = pd.DataFrame(columns=['Event', 'Date', 'Card'])

f = open("all_events_external.txt", "r")
parsed_events = f.read()
f.close()

parsed_events = parsed_events.split(',')[:-1]

print(parsed_events)

for page in all_events:
    if page in parsed_events:
        print('already parsed:' + page)
        continue
    else: 
        time.sleep(2)
        webpage = 'https://www.cagematch.net/' + page

        cagematch_html = requests.get(webpage, headers={'Accept-Encoding': 'identity'})
        cagematch_html.encoding = cagematch_html.apparent_encoding
        soup = BeautifulSoup(cagematch_html.text, features='html.parser')

        text = soup.get_text()

        x = re.split('::::', text)

        z = re.split('«', x[0])
        event = z[0].replace('\n', '')
        event = event.rstrip()

        if event != 'Card':
            print('now parsing:' + page)
            try:
                date = re.search('Broadcast date\:(\d\d.\d\d.\d\d\d\d)', z[2]).group(1)
            except AttributeError:
                date = re.search('Date\:(\d\d.\d\d.\d\d\d\d)', z[2]).group(1)

            event_titles.append(event)
            print(event)
            print(date)

            y = [item for item in x if 'defeat' in item] #DoubleDQ, DQ, time limit draw

            sub_list = ['(c)', ' - TITLE CHANGE !!!', '!!!', 'TITLE CHANGE']

            for sub in sub_list:
                y = [item.replace(sub, '') for item in y]

            splits = ['All workers', 'NotesThis text is hidden', 'Info:']

            for split in splits:
                y = [item.split(split, 1)[0] for item in y]

            for item in y:
                match_list.append(item)
                print(item)

            event_card = event_card.append({'Event': event, 'Date': date, 'Card': y}, ignore_index=True)
            
            f = open("all_events_external.txt", "a")
            f.write(page + ',')
            f.close()
        else:
            print('future event:' + page)

['?id=1&nr=309228', '?id=1&nr=310297', '?id=1&nr=310298', '?id=1&nr=309150', '?id=1&nr=309374', '?id=1&nr=309844', '?id=1&nr=308648', '?id=1&nr=306901', '?id=1&nr=308860', '?id=1&nr=253470', '?id=1&nr=308859', '?id=1&nr=255678', '?id=1&nr=305558', '?id=1&nr=308588', '?id=1&nr=253471', '?id=1&nr=307506', '?id=1&nr=308122', '?id=1&nr=305701', '?id=1&nr=306810', '?id=1&nr=307320', '?id=1&nr=305674', '?id=1&nr=306524', '?id=1&nr=306653', '?id=1&nr=305454', '?id=1&nr=305624', '?id=1&nr=306008', '?id=1&nr=296143', '?id=1&nr=305007', '?id=1&nr=302689', '?id=1&nr=305116', '?id=1&nr=305502', '?id=1&nr=304813', '?id=1&nr=304018', '?id=1&nr=303769', '?id=1&nr=300951', '?id=1&nr=304680', '?id=1&nr=300950', '?id=1&nr=303870', '?id=1&nr=303609', '?id=1&nr=300949', '?id=1&nr=303336', '?id=1&nr=300948', '?id=1&nr=303008', '?id=1&nr=300947', '?id=1&nr=302187', '?id=1&nr=300498', '?id=1&nr=301607', '?id=1&nr=254325', '?id=1&nr=301056', '?id=1&nr=298900', '?id=1&nr=300426', '?id=1&nr=299650', '?id=1&nr=3

In [107]:
event_card.to_csv('AEW_export_' + today + '.csv', index=False)

In [108]:
#event_card = pd.read_csv('AEW_export_021121.csv')
#event_card = event_card.drop(columns=['Unnamed: 0'])

In [109]:
#https://stackoverflow.com/questions/27263805/pandas-column-of-lists-create-a-row-for-each-list-element

lst_col = 'Card'

r = pd.DataFrame({
      col:np.repeat(event_card[col].values, event_card[lst_col].str.len())
      for col in event_card.columns.drop(lst_col)}
    ).assign(**{lst_col:np.concatenate(event_card[lst_col].values)})[event_card.columns]


r

,Event,Date,Card
0,AEW Dark TV Tapings - Session 2,11.09.2021,Singles MatchThunder Rosa defeats Nikita Knight
1,AEW Dark TV Tapings - Session 2,11.09.2021,Singles MatchKiera Hogan defeats Leila Grey
2,AEW Dark TV Tapings - Session 2,11.09.2021,Tag Team MatchPrivate Party (Isiah Kassidy & M...
3,AEW Dark TV Tapings - Session 2,11.09.2021,Singles MatchTen (w/The Dark Order) defeats Br...
4,AEW Dark TV Tapings - Session 2,11.09.2021,Tag Team MatchAdrian Jaoude & Cezar Bononi def...
5,AEW Dark TV Tapings - Session 2,11.09.2021,Singles MatchLance Archer defeats ???
6,AEW Dark TV Tapings - Session 2,11.09.2021,Singles MatchRicky Starks (w/Hook) defeats Dar...
7,AEW Dark TV Tapings - Session 2,11.09.2021,Tag Team MatchThe Dark Order (Evil Uno & Stu G...
8,AEW Dark TV Tapings - Session 2,11.09.2021,Singles MatchJulia Hart (w/Brian Pillman Jr. &...
9,AEW Dark TV Tapings - Session 2,11.09.2021,Tag Team MatchBear Country (Bear Boulder & Bea...


In [110]:
def match_parser(item):
    '''
    categorizes matches by number of participants and type for parsing and strips text from string
    '''
    matchguide = pd.read_csv('aew_matchguide.csv') #document i made of match types to strip that out from the text
    for i, j in matchguide.iterrows():
        if item[:j.len] == j.specific:
            item = item[j.len:]
            return item, j.type
            break
    return item, 'uncategorized'

y = [match_parser(item) for item in r.Card]

temp1 = pd.DataFrame(columns=['results', 'matchtype'], data = y)

r['result'] = temp1.results
r['matchtype'] = temp1.matchtype

r

,Event,Date,Card,result,matchtype
0,AEW Dark TV Tapings - Session 2,11.09.2021,Singles MatchThunder Rosa defeats Nikita Knight,Thunder Rosa defeats Nikita Knight,singles
1,AEW Dark TV Tapings - Session 2,11.09.2021,Singles MatchKiera Hogan defeats Leila Grey,Kiera Hogan defeats Leila Grey,singles
2,AEW Dark TV Tapings - Session 2,11.09.2021,Tag Team MatchPrivate Party (Isiah Kassidy & M...,Private Party (Isiah Kassidy & Marq Quen) defe...,tag
3,AEW Dark TV Tapings - Session 2,11.09.2021,Singles MatchTen (w/The Dark Order) defeats Br...,Ten (w/The Dark Order) defeats Brandon Gore,singles
4,AEW Dark TV Tapings - Session 2,11.09.2021,Tag Team MatchAdrian Jaoude & Cezar Bononi def...,Adrian Jaoude & Cezar Bononi defeat Jake St. P...,tag
5,AEW Dark TV Tapings - Session 2,11.09.2021,Singles MatchLance Archer defeats ???,Lance Archer defeats ???,singles
6,AEW Dark TV Tapings - Session 2,11.09.2021,Singles MatchRicky Starks (w/Hook) defeats Dar...,Ricky Starks (w/Hook) defeats Darius Lockhart,singles
7,AEW Dark TV Tapings - Session 2,11.09.2021,Tag Team MatchThe Dark Order (Evil Uno & Stu G...,The Dark Order (Evil Uno & Stu Grayson) (w/Col...,tag
8,AEW Dark TV Tapings - Session 2,11.09.2021,Singles MatchJulia Hart (w/Brian Pillman Jr. &...,Julia Hart (w/Brian Pillman Jr. & Griff Garris...,singles
9,AEW Dark TV Tapings - Session 2,11.09.2021,Tag Team MatchBear Country (Bear Boulder & Bea...,Bear Country (Bear Boulder & Bear Bronson) def...,tag


In [111]:
r.to_csv('recent_sess.csv')

In [112]:
r_master = pd.read_csv('AEW_export_r_master.csv')
r_master

,Event,Date,Card,result,matchtype
0,AEW Dark: Elevation #30,27.09.2021,Singles MatchThunder Rosa defeats Kayla Sparks...,Thunder Rosa defeats Kayla Sparks (3:11),singles
1,AEW Dark: Elevation #30,27.09.2021,Eight Man Tag Team MatchThe Dark Order (Alan A...,"The Dark Order (Alan Angels, Alex Reynolds, Jo...",tag8
2,AEW Dark: Elevation #30,27.09.2021,Three On One Handicap MatchPaul Wight defeats ...,"Paul Wight defeats CPA, RSP & VSK (3:41)",handicap3on1
3,AEW Dynamite #103 - Grand Slam,22.09.2021,Singles MatchMJF (w/Wardlow) defeats Brian Pil...,MJF (w/Wardlow) defeats Brian Pillman Jr. (w/J...,singles
4,AEW Dynamite #103 - Grand Slam,22.09.2021,Singles MatchMalakai Black defeats Cody Rhodes...,Malakai Black defeats Cody Rhodes (w/Arn Ander...,singles
...,...,...,...,...,...
2044,AEW Double Or Nothing 2019,25.05.2019,NaN,Cody (w/Brandi Rhodes) defeats Dustin Rhodes (...,singles
2045,AEW Double Or Nothing 2019,25.05.2019,NaN,The Young Bucks (Matt Jackson & Nick Jackson) ...,tag
2046,AEW Double Or Nothing 2019,25.05.2019,NaN,Chris Jericho defeats Kenny Omega (26:52),singles
2047,AEW Double Or Nothing - The Buy In,25.05.2019,NaN,Adam Page defeats Ace Romero and Billy Gunn an...,br


In [113]:
r = pd.concat([r, r_master])
r.to_csv('AEW_export_r_master.csv', index=False)

#### categorizing matches

In [144]:
r = pd.read_csv('AEW_export_r_master.csv')

In [145]:
r[r.matchtype=='uncategorized']

,Event,Date,Card,result,matchtype


In [146]:
r.matchtype.value_counts()

singles         1273
tag              582
tag6             123
tag8              32
br                13
fourway            9
tag10              9
threeway           8
threewaytag        8
handicap2on1       5
unsanctioned       5
fourwaytag         5
tag12              2
handicap3on1       2
nineway            1
sixway             1
Name: matchtype, dtype: int64

In [147]:
r = r[['Event', 'Date', 'result', 'matchtype']]

#### parsing singles matches

In [148]:
singles_matches = r[r['matchtype'] == 'singles']

singles_matches['win1'] = [singles_splitter(i)[0] for i in singles_matches['result']]
singles_matches['lose1'] = [singles_splitter(i)[1] for i in singles_matches['result']]
singles_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in singles_matches['result']]

singles_matches.win1 = [i.rstrip().lstrip() for i in singles_matches['win1'].str.replace(r"\(w\/.*?\)","")]
singles_matches.lose1 = [i.rstrip().lstrip() for i in singles_matches['lose1'].str.replace(r"\(w\/.*?\)","")]
singles_matches.lose1 = [notes_stripper(i) for i in singles_matches.lose1]

In [149]:
singles_matches[20:30]

,Event,Date,result,matchtype,win1,lose1,time
29,AEW Dark: Elevation #30,27.09.2021,Thunder Rosa defeats Kayla Sparks (3:11),singles,Thunder Rosa,Kayla Sparks,(3:11)
32,AEW Dynamite #103 - Grand Slam,22.09.2021,MJF (w/Wardlow) defeats Brian Pillman Jr. (w/J...,singles,MJF,Brian Pillman Jr.,(9:05)
33,AEW Dynamite #103 - Grand Slam,22.09.2021,Malakai Black defeats Cody Rhodes (w/Arn Ander...,singles,Malakai Black,Cody Rhodes,(11:25)
35,AEW Dynamite #103 - Grand Slam,22.09.2021,Dr. Britt Baker DMD (w/Jamie Hayter & Rebel) ...,singles,Dr. Britt Baker DMD,Ruby Soho,(13:20)
36,AEW Rampage #7 - Grand Slam,24.09.2021,CM Punk defeats Powerhouse Hobbs (w/Hook) (13:35),singles,CM Punk,Powerhouse Hobbs,(13:35)
40,AEW Rampage #7 - Grand Slam,24.09.2021,Penelope Ford (w/The Bunny) defeats Anna Jay (...,singles,Penelope Ford,Anna Jay,(6:48)
43,AEW Dark #108,14.09.2021,Anna Jay defeats Ashley D'Amboise (1:28),singles,Anna Jay,Ashley D'Amboise,(1:28)
45,AEW Dark #108,14.09.2021,Jade Cargill (w/Mark Sterling) defeats Angelic...,singles,Jade Cargill,Angelica Risk,(1:11)
46,AEW Dark #108,14.09.2021,Fuego Del Sol defeats Mysterious Movado (3:14),singles,Fuego Del Sol,Mysterious Movado,(3:14)
47,AEW Dark #108,14.09.2021,Shawn Spears defeats Khash (3:34),singles,Shawn Spears,Khash,(3:34)


#### parsing tag matches

In [150]:
tag_matches = r[r['matchtype'] == 'tag']

tag_matches['win1'] = [tag_splitter(i)[0] for i in tag_matches['result']]
tag_matches['lose1'] = [tag_splitter(i)[1] for i in tag_matches['result']]
tag_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag_matches['result']]

tag_matches.win1 = [i.rstrip().lstrip() for i in tag_matches['win1'].str.replace(r"\(w\/.*?\)","")]
tag_matches.lose1 = [i.rstrip().lstrip() for i in tag_matches['lose1'].str.replace(r"\(w\/.*?\)","")]

tag_matches['win1'] = [tag_team_parser(i) for i in tag_matches.win1]
tag_matches['lose1'] = [tag_team_parser(i) for i in tag_matches.lose1]

tag_matches[['win1','win2']] = tag_matches.win1.str.split(" & ",expand=True)
tag_matches[['lose1','lose2']] = tag_matches.lose1.str.split(" & ",expand=True) 

tag_matches = tag_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'lose1', 'lose2', 'time']]

In [151]:
tag_matches

,Event,Date,result,matchtype,win1,win2,lose1,lose2,time
2,AEW Dark TV Tapings - Session 2,11.09.2021,Private Party (Isiah Kassidy & Marq Quen) defe...,tag,Isiah Kassidy,Marq Quen,Carlie Bravo,JDX,(Unk.)
4,AEW Dark TV Tapings - Session 2,11.09.2021,Adrian Jaoude & Cezar Bononi defeat Jake St. P...,tag,Adrian Jaoude,Cezar Bononi,Jake St. Patrick,Ryzin,(Unk.)
7,AEW Dark TV Tapings - Session 2,11.09.2021,The Dark Order (Evil Uno & Stu Grayson) (w/Col...,tag,Evil Uno,Stu Grayson,Adrian Alanis,Liam Gray,(Unk.)
9,AEW Dark TV Tapings - Session 2,11.09.2021,Bear Country (Bear Boulder & Bear Bronson) def...,tag,Bear Boulder,Bear Bronson,Jameson Ryan,Rick Fulton,(Unk.)
10,AEW Dark TV Tapings - Session 2,11.09.2021,Dante Martin & Matt Sydal defeat The Factory (...,tag,Dante Martin,Matt Sydal,Aaron Solo,QT Marshall,(Unk.)
...,...,...,...,...,...,...,...,...,...
2056,AEW Fight For The Fallen,13.07.2019,The Lucha Brothers (Pentagon Jr. & Rey Fenix) ...,tag,Pentagon Jr.,Rey Fenix,Frankie Kazarian,Scorpio Sky,(15:05)
2058,AEW Fight For The Fallen,13.07.2019,The Young Bucks (Matt Jackson & Nick Jackson) ...,tag,Matt Jackson,Nick Jackson,Cody,Dustin Rhodes,(31:26)
2060,AEW Fight For The Fallen - The Buy In,13.07.2019,Bea Priestley & Shoko Nakajima defeat Dr. Brit...,tag,Bea Priestley,Shoko Nakajima,Dr. Britt Baker DMD,Riho,(15:38)
2071,AEW Double Or Nothing 2019,25.05.2019,Best Friends (Chuck Taylor & Trent) defeat Ang...,tag,Chuck Taylor,Trent,Angelico,Jack Evans,(12:42)


#### parsing tag6s

In [152]:
def tag6_parser(item):
    one_and_two = '.*? & .*? \(.*\)'
    two_and_one = '.*? \(.*\) & .*'
    trio = '.*? \(.*\)'
    three_singles = '.*? \& .*'
    
    if re.match(one_and_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[2], x[3]]
        return(z)
    elif re.match(two_and_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[4]]
        return(z)
    elif re.match(trio, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3]]
        return(z)
    elif re.match(three_singles, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[2]]
        return(z)

In [153]:
tag6_matches = r[r['matchtype'] == 'tag6']

tag6_matches['win'] = [tag_splitter(i)[0] for i in tag6_matches['result']]
tag6_matches['lose'] = [tag_splitter(i)[1] for i in tag6_matches['result']]
tag6_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag6_matches['result']]

tag6_matches.win = [i.rstrip().lstrip() for i in tag6_matches['win'].str.replace(r"\(w.*\)","")]
tag6_matches.lose = [i.rstrip().lstrip() for i in tag6_matches['lose'].str.replace(r"\(w.*\)","")]

tag6_matches.win = [tag6_parser(i) for i in tag6_matches.win]
tag6_matches.lose = [tag6_parser(i) for i in tag6_matches.lose]

tag6_matches['win1'] = [i[0] for i in tag6_matches.win]
tag6_matches['win2'] = [i[1] for i in tag6_matches.win]
tag6_matches['win3'] = [i[2] for i in tag6_matches.win]

tag6_matches['lose1'] = [i[0] for i in tag6_matches.lose]
tag6_matches['lose2'] = [i[1] for i in tag6_matches.lose]
tag6_matches['lose3'] = [i[2] for i in tag6_matches.lose]

tag6_matches = tag6_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'win3', 'lose1', 'lose2', 'lose3', 'time']]


tag6_matches

,Event,Date,result,matchtype,win1,win2,win3,lose1,lose2,lose3,time
11,AEW Dark TV Tapings - Session 2,11.09.2021,"The Pinnacle (Cash Wheeler, Dax Harwood, Shawn...",tag6,Cash Wheeler,Dax Harwood,Shawn Spears,Aaron Frye,Austin Green,Baron Black,(Unk.)
18,AEW Dark TV Tapings - Session 2,11.09.2021,"The Gunn Club (Austin Gunn, Billy Gunn & Colte...",tag6,Austin Gunn,Billy Gunn,Colten Gunn,???,???,???,(Unk.)
37,AEW Rampage #7 - Grand Slam,24.09.2021,"The Elite (Adam Cole, Matt Jackson & Nick Jack...",tag6,Adam Cole,Matt Jackson,Nick Jackson,Christian Cage,Jungle Boy,Luchasaurus,(14:38)
48,AEW Dark #108,14.09.2021,"Chuck Taylor, Orange Cassidy & Wheeler Yuta (w...",tag6,Chuck Taylor,Orange Cassidy,Wheeler Yuta,Angelico,Jack Evans,Jora Johl,(8:08)
60,AEW Dark: Elevation #29,20.09.2021,"The Gunn Club (Austin Gunn, Billy Gunn & Colte...",tag6,Austin Gunn,Billy Gunn,Colten Gunn,Anthony Bennett,Leon St. Giovanni,Ray Jaz,(Unk.)
...,...,...,...,...,...,...,...,...,...,...,...
2042,AEW All Out 2019,31.08.2019,"SCU (Christopher Daniels, Frankie Kazarian & S...",tag6,Christopher Daniels,Frankie Kazarian,Scorpio Sky,Jungle Boy,Luchasaurus,Marko Stunt,(11:40)
2052,AEW Fight For The Fallen,13.07.2019,"Maxwell Jacob Friedman, Sammy Guevara & Shawn ...",tag6,Maxwell Jacob Friedman,Sammy Guevara,Shawn Spears,Darby Allin,Jimmy Havoc,Joey Janela,(13:13)
2064,AEW Fyter Fest,29.06.2019,"The Elite (Kenny Omega, Matt Jackson & Nick Ja...",tag6,Kenny Omega,Matt Jackson,Nick Jackson,Laredo Kid,Pentagon Jr.,Rey Fenix,(20:29)
2069,AEW Double Or Nothing 2019,25.05.2019,"SCU (Christopher Daniels, Frankie Kazarian & S...",tag6,Christopher Daniels,Frankie Kazarian,Scorpio Sky,CIMA,El Lindaman,T-Hawk,(13:38)


#### parsing tag8s

In [154]:
def tag8_parser(item):
    two_and_two = '.*? \(.*\) & .*? \(.*\)'
    two_and_two_error = '.*? \(.*\)  & .*? \(.*\)'
    three_and_one = '.*? \(.*\) & .*'
    one_one_two = '.*,.* & .*? \(.*\)'
    two_one_one = '.*\(.*\),.*&.*'
    one_and_three = '.* & .*? \(.*\)'
    quad = '.*? \(.*\)'
    
    if re.match(two_and_two, item) or re.match(two_and_two_error, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[5], x[6]]
        return(z)
    elif re.match(two_one_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[4], x[5]]
        return(z)
    elif re.match(three_and_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[5]]
        return(z)
    elif re.match(one_one_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[3], x[4]]
        return(z)
    elif re.match(one_and_three, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[2], x[3], x[4]]
        return(z)
    elif re.match(quad, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4]]
        return(z)
    else:
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[2], x[3]]
        return(z)

In [155]:
tag8_matches = r[r['matchtype'] == 'tag8']

tag8_matches['win'] = [tag_splitter(i)[0] for i in tag8_matches['result']]
tag8_matches['lose'] = [tag_splitter(i)[1] for i in tag8_matches['result']]
tag8_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag8_matches['result']]

tag8_matches.win = [i.rstrip().lstrip() for i in tag8_matches['win'].str.replace(r"\(w\/.*?\)","")]
tag8_matches.lose = [i.rstrip().lstrip() for i in tag8_matches['lose'].str.replace(r"\(w\/.*?\)","")]

tag8_matches.win = [tag8_parser(i) for i in tag8_matches.win]
tag8_matches.lose = [tag8_parser(i) for i in tag8_matches.lose]

tag8_matches['win1'] = [i[0] for i in tag8_matches.win]
tag8_matches['win2'] = [i[1] for i in tag8_matches.win]
tag8_matches['win3'] = [i[2] for i in tag8_matches.win]
tag8_matches['win4'] = [i[3] for i in tag8_matches.win]

tag8_matches['lose1'] = [i[0] for i in tag8_matches.lose]
tag8_matches['lose2'] = [i[1] for i in tag8_matches.lose]
tag8_matches['lose3'] = [i[2] for i in tag8_matches.lose]
tag8_matches['lose4'] = [i[3] for i in tag8_matches.lose]


tag8_matches = tag8_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'win3', 'win4', 'lose1', 'lose2', 'lose3', 'lose4', 'time']]


tag8_matches


,Event,Date,result,matchtype,win1,win2,win3,win4,lose1,lose2,lose3,lose4,time
30,AEW Dark: Elevation #30,27.09.2021,"The Dark Order (Alan Angels, Alex Reynolds, Jo...",tag8,Alan Angels,Alex Reynolds,John Silver,Ten,Dean Alexander,Eric James,Kevin Tibbs,TJ Crawford,(5:42)
39,AEW Rampage #7 - Grand Slam,24.09.2021,The Inner Circle (Ortiz & Santana) & The Lucha...,tag8,Ortiz,Santana,Penta El Zero Miedo,Rey Fenix,Isiah Kassidy,Marq Quen,The Blade,The Butcher,(9:43)
131,AEW Dynamite #100,01.09.2021,"The Elite (Doc Gallows, Karl Anderson, Matt Ja...",tag8,Doc Gallows,Karl Anderson,Matt Jackson,Nick Jackson,Jungle Boy,Luchasaurus,Penta El Zero Miedo,Rey Fenix,(10:13)
196,AEW Dark #103,17.08.2021,Jurassic Express (Jungle Boy & Luchasaurus) & ...,tag8,Jungle Boy,Luchasaurus,Brian Pillman Jr.,Griff Garrison,Cezar Bononi,JD Drake,Peter Avalon,Ryan Nemeth,(8:14)
272,AEW Dark #100,03.08.2021,The Acclaimed (Anthony Bowens & Max Caster) & ...,tag8,Anthony Bowens,Max Caster,Matt Hardy,The Blade,Matt Sydal,Mike Sydal,Brian Pillman Jr.,Griff Garrison,(5:48)
327,AEW Dark #98,20.07.2021,The Acclaimed (Anthony Bowens & Max Caster) & ...,tag8,Anthony Bowens,Max Caster,Isiah Kassidy,Marq Quen,Alan Angels,Ten,Brian Pillman Jr.,Griff Garrison,(6:52)
355,AEW Dark: Elevation #18,12.07.2021,The Gunn Club (Billy Gunn & Colten Gunn) & The...,tag8,Billy Gunn,Colten Gunn,Brian Pillman Jr.,Griff Garrison,Anthony Bowens,Max Caster,Luther,Serpentico,(6:21)
383,AEW Dark #96,06.07.2021,"The Dark Order (Alan Angels, Alex Reynolds, Co...",tag8,Alan Angels,Alex Reynolds,Colt Cabana,Ten,Chandler Hopkins,Dean Alexander,Jason Hotch,Will Allday,(4:03)
711,AEW The House Always Wins,09.04.2021,"The Pinnacle (Cash Wheeler, Dax Harwood, MJF &...",tag8,Cash Wheeler,Dax Harwood,MJF,Shawn Spears,Christopher Daniels,Dante Martin,Jungle Boy,Luchasaurus,(14:30)
716,AEW The House Always Wins,09.04.2021,"Hikaru Shida, Red Velvet, Ryo Mizunami & Tay C...",tag8,Hikaru Shida,Red Velvet,Ryo Mizunami,Tay Conti,Dr. Britt Baker DMD,Nyla Rose,Rebel,The Bunny,(14:00)


#### parsing tag10s

In [156]:
def tag10_parser(item):
    one_and_four = '.* & .*? \(.*\)'
    four_and_one = '.*? \(.*\) & .*'
    three_one_one = '.*? \(.*\), .*? & .*'
    two_one_one_one = '.*? \(.*\), .*?, .*? & .*'
    two_two_one = '.*? \(.*\), .*? \(.*\) & .*'
    two_three = '.*? \(.*?\) & .*?\(.*?\)'
    fivesome = '.*? \(.*\)'
    one_two_two = '.*?, .*? \(.*\) &.*? \(.*\)'
    
    if re.match(two_two_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[5], x[6], x[8]]
        return(z)
    elif re.match(two_one_one_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[4], x[5], x[6]]
        return(z)
    elif re.match(one_two_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[2], x[3], x[6], x[7]]
        return(z)
    elif re.match(three_one_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[5], x[6]]
        return(z)
    elif re.match(two_three, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[5], x[6], x[7]]
        return(z)
    elif re.match(four_and_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[6]]
        return(z)
    elif re.match(one_and_four, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[2], x[3], x[4], x[5]]
        return(z)
    elif re.match(fivesome, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[5]]
        return(z)
    else:
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[2], x[3], x[4]]
        return(z)

In [157]:
tag10_matches = r[r['matchtype'] == 'tag10']

tag10_matches['win'] = [tag_splitter(i)[0] for i in tag10_matches['result']]
tag10_matches['lose'] = [tag_splitter(i)[1] for i in tag10_matches['result']]
tag10_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag10_matches['result']]

tag10_matches.win = [i.rstrip().lstrip() for i in tag10_matches['win'].str.replace(r"\(w\/.*?\)","")]
tag10_matches.lose = [i.rstrip().lstrip() for i in tag10_matches['lose'].str.replace(r"\(w\/.*?\)","")]

tag10_matches.win = [tag10_parser(i) for i in tag10_matches.win]
tag10_matches.lose = [tag10_parser(i) for i in tag10_matches.lose]

tag10_matches['win1'] = [i[0] for i in tag10_matches.win]
tag10_matches['win2'] = [i[1] for i in tag10_matches.win]
tag10_matches['win3'] = [i[2] for i in tag10_matches.win]
tag10_matches['win4'] = [i[3] for i in tag10_matches.win]
tag10_matches['win5'] = [i[4] for i in tag10_matches.win]

tag10_matches['lose1'] = [i[0] for i in tag10_matches.lose]
tag10_matches['lose2'] = [i[1] for i in tag10_matches.lose]
tag10_matches['lose3'] = [i[2] for i in tag10_matches.lose]
tag10_matches['lose4'] = [i[3] for i in tag10_matches.lose]
tag10_matches['lose5'] = [i[4] for i in tag10_matches.lose]

tag10_matches = tag10_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'win3', 'win4', 'win5', 'lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'time']]

tag10_matches

,Event,Date,result,matchtype,win1,win2,win3,win4,win5,lose1,lose2,lose3,lose4,lose5,time
97,AEW All Out 2021 - The Buy In,05.09.2021,"Chuck Taylor, Jurassic Express (Jungle Boy & L...",tag10,Jurassic Express,Jungle Boy,Luchasaurus,Orange Cassidy,Wheeler Yuta,Angelico,Isiah Kassidy,Jack Evans,Marq Quen,Matt Hardy,(9:26)
277,AEW Dynamite #96 - Fight For The Fallen 2021,28.07.2021,"The Elite (Doc Gallows, Karl Anderson, Kenny O...",tag10,Doc Gallows,Karl Anderson,Kenny Omega,Matt Jackson,Nick Jackson,Adam Page,Alex Reynolds,Evil Uno,John Silver,Stu Grayson,am Page
537,AEW Double Or Nothing 2021,30.05.2021,"The Inner Circle (Chris Jericho, Jake Hager, O...",tag10,Chris Jericho,Jake Hager,Ortiz,Sammy Guevara,Santana,Cash Wheeler,Dax Harwood,MJF,Shawn Spears,Wardlow,(32:26)
645,AEW Dynamite #84 - Blood & Guts,05.05.2021,"The Pinnacle (Cash Wheeler, Dax Harwood, MJF, ...",tag10,Cash Wheeler,Dax Harwood,MJF,Shawn Spears,Wardlow,Chris Jericho,Jake Hager,Ortiz,Sammy Guevara,Santana,(17:00)
717,AEW The House Always Wins,09.04.2021,"Death Triangle (PAC, Penta El Zero Miedo & Rey...",tag10,PAC,Penta El Zero Miedo,Rey Fenix,Matt Sydal,Mike Sydal,Matt Jackson,Nick Jackson,Kenny Omega,Konosuke Takeshita,Michael Nakazawa,(12:25)
855,AEW Dynamite #77 - St. Patrick's Day Slam,17.03.2021,"Matt Hardy, Private Party (Isiah Kassidy & Mar...",tag10,Matt Hardy,Isiah Kassidy,Marq Quen,The Blade,The Butcher,Bear Boulder,Bear Bronson,Jungle Boy,Luchasaurus,Marko Stunt,(8:47)
1116,AEW Dark #70,12.01.2021,"The Dark Order (Colt Cabana, Evil Uno, Five, S...",tag10,Colt Cabana,Evil Uno,Five,Stu Grayson,Ten,Adam Priest,Danny Limelight,Shawn Dean,Vary Morales,Zack Clayton,(8:19)
1561,AEW Dynamite #43,29.07.2020,"Best Friends (Chuck Taylor & Trent), The Juras...",tag10,Chuck Taylor,Trent,Jungle Boy,Luchasaurus,Orange Cassidy,Chris Jericho,Jake Hager,Ortiz,Sammy Guevara,Santana,(12:06)
1710,AEW Double Or Nothing 2020,23.05.2020,"The Elite (Adam Page, Kenny Omega, Matt Jackso...",tag10,Adam Page,Kenny Omega,Matt Jackson,Nick Jackson,Matt Hardy,Chris Jericho,Jake Hager,Ortiz,Sammy Guevara,Santana,(33:59)


#### parsing tag12s

In [158]:
def tag12_parser(item):
    four_and_two = '.*? \(.*\) & .* \(.*\)'
    five_and_one = '.*? \(.*\) & .*'
    two_two_two =  '.*? \(.*\), .*? \(.*\) & .*? \(.*\)'
    sixsome = '.*? \(.*\)'
    
    if re.match(two_two_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[5], x[6], x[9], x[10]]
        return(z)
    elif re.match(four_and_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[7], x[8]]
        return(z)
    elif re.match(five_and_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[5], x[7]]
        return(z)
    elif re.match(sixsome, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[5], x[6]]
        return(z)
    else:
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[2], x[3], x[4]]
        return(z)

In [159]:
tag12_matches = r[r['matchtype'] == 'tag12']

tag12_matches['win'] = [tag_splitter(i)[0] for i in tag12_matches['result']]
tag12_matches['lose'] = [tag_splitter(i)[1] for i in tag12_matches['result']]
tag12_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag12_matches['result']]

tag12_matches.win = [i.rstrip().lstrip() for i in tag12_matches['win'].str.replace(r"\(w\/.*?\)","")]
tag12_matches.lose = [i.rstrip().lstrip() for i in tag12_matches['lose'].str.replace(r"\(w\/.*?\)","")]

tag12_matches.win = [tag12_parser(i) for i in tag12_matches.win]
tag12_matches.lose = [tag12_parser(i) for i in tag12_matches.lose]

tag12_matches['win1'] = [i[0] for i in tag12_matches.win]
tag12_matches['win2'] = [i[1] for i in tag12_matches.win]
tag12_matches['win3'] = [i[2] for i in tag12_matches.win]
tag12_matches['win4'] = [i[3] for i in tag12_matches.win]
tag12_matches['win5'] = [i[4] for i in tag12_matches.win]
tag12_matches['win6'] = [i[5] for i in tag12_matches.win]

tag12_matches['lose1'] = [i[0] for i in tag12_matches.lose]
tag12_matches['lose2'] = [i[1] for i in tag12_matches.lose]
tag12_matches['lose3'] = [i[2] for i in tag12_matches.lose]
tag12_matches['lose4'] = [i[3] for i in tag12_matches.lose]
tag12_matches['lose5'] = [i[4] for i in tag12_matches.lose]
tag12_matches['lose6'] = [i[5] for i in tag12_matches.lose]

tag12_matches = tag12_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'win3', 'win4', 'win5', 'win6', 'lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'lose6', 'time']]


tag12_matches

,Event,Date,result,matchtype,win1,win2,win3,win4,win5,win6,lose1,lose2,lose3,lose4,lose5,lose6,time
1181,AEW Dynamite #64,16.12.2020,"The Inner Circle (Chris Jericho, Jake Hager, M...",tag12,Chris Jericho,Jake Hager,MJF,Ortiz,Sammy Guevara,Santana,Chuck Taylor,Trent,Brian Pillman Jr.,Griff Garrison,Darius Martin,Daunte Martin,(14:19)
1547,AEW Dynamite #44,05.08.2020,"The Dark Order (Evil Uno, Five, Mr. Brodie Lee...",tag12,Evil Uno,Five,Mr. Brodie Lee,Nine,Stu Grayson,Colt Cabana,Adam Page,Kenny Omega,Matt Jackson,Nick Jackson,Cash Wheeler,Dax Harwood,(17:19)


#### parsing fourways

In [160]:
fourway_matches = r[r['matchtype'] == 'fourway']


fourway_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in fourway_matches['result']]
fourway_matches.result = [i.rstrip().lstrip() for i in fourway_matches['result'].str.replace(r"\(w\/.*?\)","")]

fourway_matches['win1'] = [singles_splitter(i)[0] for i in fourway_matches['result']]
fourway_matches['lose'] = [singles_splitter(i)[1] for i in fourway_matches['result']]
fourway_matches['lose1'] = [multiway_splitter(i)[0] for i in fourway_matches.lose]
fourway_matches['lose2'] = [multiway_splitter(i)[1] for i in fourway_matches.lose]
fourway_matches['lose3'] = [multiway_splitter(i)[2] for i in fourway_matches.lose]


fourway_matches = fourway_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', 'lose3', 'time']]


fourway_matches

,Event,Date,result,matchtype,win1,lose1,lose2,lose3,time
1143,AEW Dark #69,05.01.2021,Frankie Kazarian defeats Angelico and Darius...,fourway,Frankie Kazarian,Angelico,Darius Martin,Griff Garrison,(7:43)
1386,AEW Dark #56,13.10.2020,Jungle Boy defeats Evil Uno and Frankie Kazari...,fourway,Jungle Boy,Evil Uno,Frankie Kazarian,The Blade,(8:29)
1728,AEW Dynamite #32,13.05.2020,Hikaru Shida defeats Dr. Britt Baker DMD and K...,fourway,Hikaru Shida,Dr. Britt Baker DMD,Kris Statlander,Penelope Ford,(10:12)
1811,AEW Dynamite #24,18.03.2020,Hikaru Shida defeats Kris Statlander and Penel...,fourway,Hikaru Shida,Kris Statlander,Penelope Ford,Riho,(5:49)
1850,AEW Dynamite #21,26.02.2020,Hikaru Shida defeats Big Swole and Shanna and ...,fourway,Hikaru Shida,Big Swole,Shanna,Yuka Sakazaki,(9:13)
1916,AEW Dynamite #13 - Homecoming Edition,01.01.2020,Riho defeats Dr. Britt Baker DMD and Hikaru S...,fourway,Riho,Dr. Britt Baker DMD,Hikaru Shida,Nyla Rose,(9:49)
2014,AEW Dark #4,29.10.2019,Emi Sakura defeats Allie and Penelope Ford and...,fourway,Emi Sakura,Allie,Penelope Ford,Sadie Gibbs,(11:54)
2063,AEW Fyter Fest,29.06.2019,Adam Page defeats Jimmy Havoc and Jungle Boy ...,fourway,Adam Page,Jimmy Havoc,Jungle Boy,Maxwell Jacob Friedman,(10:51)
2070,AEW Double Or Nothing 2019,25.05.2019,Dr. Britt Baker DMD defeats Awesome Kong and ...,fourway,Dr. Britt Baker DMD,Awesome Kong,Kylie Rae,Nyla Rose,(11:08)


#### parsing threeways

In [161]:
threeway_matches = r[r['matchtype'] == 'threeway']


threeway_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in threeway_matches['result']]
threeway_matches.result = [i.rstrip().lstrip() for i in threeway_matches['result'].str.replace(r"\(w\/.*?\)","")]

threeway_matches['win1'] = [singles_splitter(i)[0] for i in threeway_matches['result']]
threeway_matches['lose'] = [singles_splitter(i)[1] for i in threeway_matches['result']]

threeway_matches['lose1'] = [multiway_splitter(i)[0] for i in threeway_matches.lose]
threeway_matches['lose2'] = [multiway_splitter(i)[1] for i in threeway_matches.lose]

threeway_matches = threeway_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', 'time']]


threeway_matches

,Event,Date,result,matchtype,win1,lose1,lose2,time
536,AEW Double Or Nothing 2021,30.05.2021,Kenny Omega defeats Orange Cassidy and PAC (...,threeway,Kenny Omega,Orange Cassidy,PAC,(27:10)
1800,AEW @The BTE Compound,13.04.2020,Scorpio Sky defeats Brandon Cutler and Peter A...,threeway,Scorpio Sky,Brandon Cutler,Peter Avalon,(Unk.)
1953,AEW Dark #10,10.12.2019,Scorpio Sky defeats Jimmy Havoc and Peter Aval...,threeway,Scorpio Sky,Jimmy Havoc,Peter Avalon,(6:28)
1977,AEW Dynamite #7,13.11.2019,Darby Allin defeats Peter Avalon and Shawn Sp...,threeway,Darby Allin,Peter Avalon,Shawn Spears,(3:44)
2015,AEW Dark #4,29.10.2019,Darby Allin defeats Jack Evans and Jimmy Havoc...,threeway,Darby Allin,Jack Evans,Jimmy Havoc,(13:12)
2030,AEW Dark #2,15.10.2019,Kip Sabian defeats Peter Avalon and Sonny Kis...,threeway,Kip Sabian,Peter Avalon,Sonny Kiss,(6:06)
2044,AEW All Out 2019,31.08.2019,Jimmy Havoc defeats Darby Allin and Joey Janel...,threeway,Jimmy Havoc,Darby Allin,Joey Janela,(14:57)
2062,AEW Fyter Fest,29.06.2019,Riho defeats Nyla Rose and Yuka Sakazaki (13:02),threeway,Riho,Nyla Rose,Yuka Sakazaki,(13:02)


#### parsing fourwaytags

In [162]:
fourwaytag_matches = r[r['matchtype'] == 'fourwaytag']
fourwaytag_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in fourwaytag_matches['result']]
fourwaytag_matches.result = [i.rstrip().lstrip() for i in fourwaytag_matches['result'].str.replace(r"\(w\/.*?\)","")]

fourwaytag_matches['win_team'] = [tag_splitter(i)[0] for i in fourwaytag_matches['result']]
fourwaytag_matches['lose_teams'] = [tag_splitter(i)[1] for i in fourwaytag_matches['result']]

fourwaytag_matches["loseteam1"] = np.nan
fourwaytag_matches["loseteam2"] = np.nan
fourwaytag_matches["loseteam3"] = np.nan

fourwaytag_matches[['loseteam1', 'loseteam2', 'loseteam3']] = [(multiway_splitter(i)[0], multiway_splitter(i)[1], multiway_splitter(i)[2]) for i in fourwaytag_matches.lose_teams]

fourwaytag_matches['win_team'] = [tag_team_parser(i) for i in fourwaytag_matches.win_team]
fourwaytag_matches['loseteam1'] = [tag_team_parser(i) for i in fourwaytag_matches.loseteam1]
fourwaytag_matches['loseteam2'] = [tag_team_parser(i) for i in fourwaytag_matches.loseteam2]
fourwaytag_matches['loseteam3'] = [tag_team_parser(i) for i in fourwaytag_matches.loseteam3]

fourwaytag_matches[['win1', 'win2']] = fourwaytag_matches.win_team.str.split(" & ",expand=True)
fourwaytag_matches[['lose1', 'lose2']] = fourwaytag_matches.loseteam1.str.split(" & ",expand=True) 
fourwaytag_matches[['lose3', 'lose4']] = fourwaytag_matches.loseteam2.str.split(" & ",expand=True) 
fourwaytag_matches[['lose5', 'lose6']] = fourwaytag_matches.loseteam3.str.split(" & ",expand=True) 

fourwaytag_matches = fourwaytag_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'lose6', 'time']]


fourwaytag_matches

,Event,Date,result,matchtype,win1,win2,lose1,lose2,lose3,lose4,lose5,lose6,time
644,AEW Dynamite #84 - Blood & Guts,05.05.2021,SCU (Christopher Daniels & Frankie Kazarian) d...,fourwaytag,Christopher Daniels,Frankie Kazarian,Brian Pillman Jr.,Griff Garrison,Jungle Boy,Luchasaurus,Anthony Bowens,Max Caster,(9:12)
1345,AEW Dynamite #56,21.10.2020,The Young Bucks (Matt Jackson & Nick Jackson) ...,fourwaytag,Matt Jackson,Nick Jackson,Isiah Kassidy,Marq Quen,The Blade,The Butcher,Alex Reynolds,John Silver,(13:29)
1496,AEW Dynamite #47 - Thursday Night Dynamite,27.08.2020,FTR (Cash Wheeler & Dax Harwood) defeat Best ...,fourwaytag,Cash Wheeler,Dax Harwood,Chuck Taylor,Trent,Dustin Rhodes,QT Marshall,Matt Jackson,Nick Jackson,(5:25)
1896,AEW Dynamite #15 - Bash At The Beach,15.01.2020,Adam Page & Kenny Omega defeat Best Friends (C...,fourwaytag,Adam Page,Kenny Omega,Chuck Taylor,Trent,Ortiz,Santana,Matt Jackson,Nick Jackson,(16:35)
1935,AEW Dark #12,24.12.2019,Proud-N-Powerful (Ortiz & Santana) defeat Best...,fourwaytag,Ortiz,Santana,Chuck Taylor,Trent,Isiah Kassidy,Marq Quen,Angelico,Jack Evans,(16:08)


#### parsing threewaytags

In [163]:
threewaytag_matches = r[r['matchtype'] == 'threewaytag']
threewaytag_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in threewaytag_matches['result']]
threewaytag_matches.result = [i.rstrip().lstrip() for i in threewaytag_matches['result'].str.replace(r"\(w\/.*?\)","")]

threewaytag_matches["loseteam1"] = np.nan
threewaytag_matches["loseteam2"] = np.nan

threewaytag_matches['win_team'] = [tag_splitter(i)[0] for i in threewaytag_matches['result']]
threewaytag_matches['lose_teams'] = [tag_splitter(i)[1] for i in threewaytag_matches['result']]
threewaytag_matches[['loseteam1', 'loseteam2']] = [(multiway_splitter(i)[0], multiway_splitter(i)[1]) for i in threewaytag_matches.lose_teams]

threewaytag_matches['win_team'] = [tag_team_parser(i) for i in threewaytag_matches.win_team]
threewaytag_matches['loseteam1'] = [tag_team_parser(i) for i in threewaytag_matches.loseteam1]
threewaytag_matches['loseteam2'] = [tag_team_parser(i) for i in threewaytag_matches.loseteam2]

threewaytag_matches[['win1', 'win2']] = threewaytag_matches.win_team.str.split(" & ",expand=True)
threewaytag_matches[['lose1', 'lose2']] = threewaytag_matches.loseteam1.str.split(" & ",expand=True) 
threewaytag_matches[['lose3', 'lose4']] = threewaytag_matches.loseteam2.str.split(" & ",expand=True) 

threewaytag_matches = threewaytag_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'lose1', 'lose2', 'lose3', 'lose4', 'time']]

threewaytag_matches

,Event,Date,result,matchtype,win1,win2,lose1,lose2,lose3,lose4,time
1007,AEW Dark #76,23.02.2021,SCU (Christopher Daniels & Frankie Kazarian) d...,threewaytag,Christopher Daniels,Frankie Kazarian,Angelico,Jack Evans,Matt Sydal,Mike Sydal,(11:38)
1071,AEW Dynamite #69,20.01.2021,The Inner Circle (Chris Jericho & MJF) defeat...,threewaytag,Chris Jericho,MJF,Jake Hager,Sammy Guevara,Ortiz,Santana,(12:38)
1248,AEW Dark #64,01.12.2020,The Jurassic Express (Jungle Boy & Luchasaurus...,threewaytag,Jungle Boy,Luchasaurus,Five,Ten,Joey Janela,Sonny Kiss,(9:35)
1270,AEW Dark #63,24.11.2020,The Dark Order (Evil Uno & Stu Grayson) defeat...,threewaytag,Evil Uno,Stu Grayson,Luther,Serpentico,Joey Janela,Sonny Kiss,(9:02)
1596,AEW Dark #44,28.07.2020,The Inner Circle (Ortiz & Santana) defeat SCU ...,threewaytag,Ortiz,Santana,Christopher Daniels,Frankie Kazarian,Isiah Kassidy,Marq Quen,(11:01)
1987,AEW Full Gear 2019,09.11.2019,SCU (Frankie Kazarian & Scorpio Sky) defeat T...,threewaytag,Frankie Kazarian,Scorpio Sky,Pentagon Jr.,Rey Fenix,Isiah Kassidy,Marq Quen,(13:07)
2054,AEW Fight For The Fallen,13.07.2019,The Dark Order (Evil Uno & Stu Grayson) defeat...,threewaytag,Evil Uno,Stu Grayson,Jungle Boy,Luchasaurus,Angelico,Jack Evans,(15:07)
2066,AEW Fyter Fest - The Buy In,29.06.2019,Best Friends (Chuck Taylor & Trent) defeat SCU...,threewaytag,Chuck Taylor,Trent,Frankie Kazarian,Scorpio Sky,Isiah Kassidy,Marq Quen,(15:56)


#### parsing handicaps

In [164]:
handicap3on1_matches = r[r['matchtype'] == 'handicap3on1']

handicap3on1_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in handicap3on1_matches['result']]
handicap3on1_matches.result = [i.rstrip().lstrip() for i in handicap3on1_matches['result'].str.replace(r"\(w\/.*?\)","")]

handicap3on1_matches['win1'] = [singles_splitter(i)[0] for i in handicap3on1_matches['result']]
handicap3on1_matches['lose_teams'] = [singles_splitter(i)[1] for i in handicap3on1_matches['result']]

handicap3on1_matches['lose1'] = np.nan
handicap3on1_matches['lose2'] = np.nan
handicap3on1_matches['lose3'] = np.nan

handicap3on1_matches[['lose1', 'lose2', 'lose3']] = [re.split(r'[,&()]\s*', i) for i in handicap3on1_matches.lose_teams]

handicap3on1_matches = handicap3on1_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', 'lose3']]


handicap3on1_matches

,Event,Date,result,matchtype,win1,lose1,lose2,lose3
31,AEW Dark: Elevation #30,27.09.2021,"Paul Wight defeats CPA, RSP & VSK (3:41)",handicap3on1,Paul Wight,CPA,RSP,VSK
1499,AEW Dynamite #47 - Thursday Night Dynamite,27.08.2020,"Big Swole defeats Dr. Britt Baker DMD, Penelop...",handicap3on1,Big Swole,Dr. Britt Baker DMD,Penelope Ford,Reba


In [165]:
handicap2on1_matches = r[r['matchtype'] == 'handicap2on1']

handicap2on1_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in handicap2on1_matches['result']]
handicap2on1_matches.result = [i.rstrip().lstrip() for i in handicap2on1_matches['result'].str.replace(r"\(w\/.*?\)","")]

handicap2on1_matches['win'] = [singles_splitter(i)[0] for i in handicap2on1_matches['result']]
handicap2on1_matches['win_teams'] = [tag_splitter(i)[0] for i in handicap2on1_matches['result']]
handicap2on1_matches['winteam1'] = [tag_team_parser(i) for i in handicap2on1_matches.win_teams]
handicap2on1_matches[['win1', 'win2']] = handicap2on1_matches.winteam1.str.split(" & ",expand=True) 
handicap2on1_matches['lose_teams'] = [tag_splitter(i)[1] for i in handicap2on1_matches['result']]
handicap2on1_matches['loseteam1'] = [tag_team_parser(i) for i in handicap2on1_matches.lose_teams]

handicap2on1_matches['lose1'] = np.nan
handicap2on1_matches['lose2'] = np.nan

handicap2on1_matches[['lose1', 'lose2']] = handicap2on1_matches.loseteam1.str.split(" & ",expand=True) 
handicap2on1_matches['lose1'] = [i[2:] if i[:2] == 's ' else i for i in handicap2on1_matches.lose1]

handicap2on1_matches = handicap2on1_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'lose1', 'lose2', 'time']]

handicap2on1_matches

,Event,Date,result,matchtype,win1,win2,lose1,lose2,time
125,AEW Rampage #4,03.09.2021,Kris Statlander defeats Jamie Hayter & Rebel ...,handicap2on1,Kris Statlander,None,Jamie Hayter,Rebel,(4:11)
454,AEW Dynamite #90,18.06.2021,The Men Of The Year (Ethan Page & Scorpio Sky)...,handicap2on1,Ethan Page,Scorpio Sky,Darby Allin,None,(Unk.)
1544,AEW Dark #47,18.08.2020,Lance Archer defeats Jessy Sorensen & Jon Cru...,handicap2on1,Lance Archer,None,Jessy Sorensen,Jon Cruz,(2:42)
1613,AEW Dynamite #40 - Fyter Fest 2020 - Tag 2,08.07.2020,Nyla Rose defeats Kenzie Paige & KiLynn King (...,handicap2on1,Nyla Rose,None,Kenzie Paige,KiLynn King,(2:21)
1833,AEW Dynamite #22,04.03.2020,The Inner Circle (Chris Jericho & Sammy Guevar...,handicap2on1,Chris Jericho,Sammy Guevara,Darby Allin,None,(Unk.)


#### other parses

In [166]:
nineway_matches = r[r['matchtype'] == 'nineway']


nineway_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in nineway_matches['result']]
nineway_matches.result = [i.rstrip().lstrip() for i in nineway_matches['result'].str.replace(r"\(w\/.*?\)","")]

nineway_matches['win1'] = [singles_splitter(i)[0] for i in nineway_matches['result']]
nineway_matches['lose'] = [singles_splitter(i)[1] for i in nineway_matches['result']]

nineway_matches['lose1'] = np.nan
nineway_matches['lose2'] = np.nan
nineway_matches['lose3'] = np.nan
nineway_matches['lose4'] = np.nan
nineway_matches['lose5'] = np.nan
nineway_matches['lose6'] = np.nan
nineway_matches['lose7'] = np.nan
nineway_matches['lose8'] = np.nan

nineway_matches[['lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'lose6', 'lose7', 'lose8']] = \
[(multiway_splitter(i)[0], multiway_splitter(i)[1], multiway_splitter(i)[2], multiway_splitter(i)[3], \
  multiway_splitter(i)[4], multiway_splitter(i)[5], multiway_splitter(i)[6], multiway_splitter(i)[7], \
 ) for i in nineway_matches.lose]


nineway_matches = nineway_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', \
                                   'lose3', 'lose4', 'lose5', 'lose6', 'lose7', 'lose8', 'time']]


nineway_matches

,Event,Date,result,matchtype,win1,lose1,lose2,lose3,lose4,lose5,lose6,lose7,lose8,time
1703,AEW Double Or Nothing 2020,23.05.2020,Brian Cage defeats Colt Cabana and Darby Alli...,nineway,Brian Cage,Colt Cabana,Darby Allin,Frankie Kazarian,Kip Sabian,Joey Janela,Luchasaurus,Orange Cassidy,Scorpio Sky,(27:28)


In [167]:
sixway_matches = r[r['matchtype'] == 'sixway']


sixway_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in sixway_matches['result']]
sixway_matches.result = [i.rstrip().lstrip() for i in sixway_matches['result'].str.replace(r"\(w\/.*?\)","")]

sixway_matches['win1'] = [singles_splitter(i)[0] for i in sixway_matches['result']]
sixway_matches['lose'] = [singles_splitter(i)[1] for i in sixway_matches['result']]

sixway_matches['lose1'] = np.nan
sixway_matches['lose2'] = np.nan
sixway_matches['lose3'] = np.nan
sixway_matches['lose4'] = np.nan
sixway_matches['lose5'] = np.nan
sixway_matches['lose6'] = np.nan

sixway_matches[['lose1', 'lose2', 'lose3', 'lose4', 'lose5']] = \
[(multiway_splitter(i)[0], multiway_splitter(i)[1], multiway_splitter(i)[2], multiway_splitter(i)[3], \
  multiway_splitter(i)[4]) for i in sixway_matches.lose]


sixway_matches = sixway_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', \
                                   'lose3', 'lose4', 'lose5', 'time']]


sixway_matches

,Event,Date,result,matchtype,win1,lose1,lose2,lose3,lose4,lose5,time
922,AEW Revolution 2021,07.03.2021,Scorpio Sky defeats Cody Rhodes and Ethan Pag...,sixway,Scorpio Sky,Cody Rhodes,Ethan Page,Lance Archer,Max Caster,Penta El Zero Miedo,(23:07)


### merge

In [168]:
data_frames = [singles_matches, tag_matches, tag6_matches, tag8_matches, tag10_matches, tag12_matches, fourway_matches,\
               threeway_matches, threewaytag_matches, fourwaytag_matches, handicap3on1_matches, handicap2on1_matches,\
               sixway_matches, nineway_matches]

merged_df = pd.concat(data_frames)
merged_df[merged_df.columns] = merged_df.apply(lambda x: x.str.strip())
merged_df['win1'] = [re.sub('\(vakant\)', '', x) for x in merged_df.win1]

merged_df = merged_df[['Event', 'Date', 'result', 'matchtype', 'time',\
                     'win1', 'win2', 'win3', 'win4', 'win5', 'win6',\
                     'lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'lose6', 'lose7', 'lose8']]

draws = pd.read_csv('all_draws.csv')
merged_df = pd.concat([merged_df, draws])

merged_df.to_csv('merged_' + today + '.csv')

In [169]:
merged_df

,Event,Date,result,matchtype,time,win1,win2,win3,win4,win5,win6,lose1,lose2,lose3,lose4,lose5,lose6,lose7,lose8
0,AEW Dark TV Tapings - Session 2,11.09.2021,Thunder Rosa defeats Nikita Knight,singles,(Unk.),Thunder Rosa,NaN,NaN,NaN,NaN,NaN,Nikita Knight,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AEW Dark TV Tapings - Session 2,11.09.2021,Kiera Hogan defeats Leila Grey,singles,(Unk.),Kiera Hogan,NaN,NaN,NaN,NaN,NaN,Leila Grey,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AEW Dark TV Tapings - Session 2,11.09.2021,Ten (w/The Dark Order) defeats Brandon Gore,singles,(Unk.),Ten,NaN,NaN,NaN,NaN,NaN,Brandon Gore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AEW Dark TV Tapings - Session 2,11.09.2021,Lance Archer defeats ???,singles,(Unk.),Lance Archer,NaN,NaN,NaN,NaN,NaN,???,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AEW Dark TV Tapings - Session 2,11.09.2021,Ricky Starks (w/Hook) defeats Darius Lockhart,singles,(Unk.),Ricky Starks,NaN,NaN,NaN,NaN,NaN,Darius Lockhart,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,AEW Dynamite #4,23.10.2019,Jon Moxley vs. PAC - Time Limit Draw (12:10),draw,(12:10),Jon Moxley,NaN,NaN,NaN,NaN,NaN,PAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AEW Dynamite #55 - Anniversary Show,14.10.2020,Cody (w/Arn Anderson & Brandi Rhodes) (c) vs. ...,draw,(20:00),Cody,NaN,NaN,NaN,NaN,NaN,Orange Cassidy,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AEW Dynamite #12,18.12.2019,Chris Jericho (w/Jake Hager) vs. Jungle Boy (w...,draw,(10:00),Chris Jericho,NaN,NaN,NaN,NaN,NaN,Jungle Boy,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AEW Dynamite #85,12.05.2021,Orange Cassidy vs. PAC - Double KO (14:02),draw,(14:02),PAC,NaN,NaN,NaN,NaN,NaN,Orange Cassidy,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Trueskill

In [170]:
alias_dict = pd.read_excel('alias_dict.xlsx')
aliases = list(alias_dict.alias)

def alias_collapser(name):
    if name in aliases:
        y = list(alias_dict[alias_dict['alias'] == name]['name'])
        return y[0]
    else:
        return name

In [171]:
all_matches = pd.read_csv('merged_' + today + '.csv')
all_matches = all_matches.drop(columns=['Unnamed: 0'])
all_matches['Date'] = pd.to_datetime(all_matches.Date, format='%d.%m.%Y')
all_matches = all_matches.sort_values('Date')


all_matches.tail(50)

,Event,Date,result,matchtype,time,win1,win2,win3,win4,win5,win6,lose1,lose2,lose3,lose4,lose5,lose6,lose7,lose8
4,AEW Dark TV Tapings - Session 2,2021-09-11,Ricky Starks (w/Hook) defeats Darius Lockhart,singles,(Unk.),Ricky Starks,NaN,NaN,NaN,NaN,NaN,Darius Lockhart,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1279,AEW Dark TV Tapings - Session 2,2021-09-11,The Varsity Blonds (Brian Pillman Jr. & Griff ...,tag,(Unk.),Brian Pillman Jr.,Griff Garrison,NaN,NaN,NaN,NaN,Peter Avalon,Ryan Nemeth,NaN,NaN,NaN,NaN,NaN,NaN
1,AEW Dark TV Tapings - Session 2,2021-09-11,Kiera Hogan defeats Leila Grey,singles,(Unk.),Kiera Hogan,NaN,NaN,NaN,NaN,NaN,Leila Grey,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,AEW Dark: Elevation #28,2021-09-13,Emi Sakura (w/Lulu Pencil) defeats Queen Amina...,singles,(3:41),Emi Sakura,NaN,NaN,NaN,NaN,NaN,Queen Aminata,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,AEW Dark: Elevation #28,2021-09-13,Frankie Kazarian defeats Ren Jones (2:59),singles,(2:59),Frankie Kazarian,NaN,NaN,NaN,NaN,NaN,Ren Jones,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,AEW Dark: Elevation #28,2021-09-13,Penelope Ford (w/The Bunny) defeats Layna Lenn...,singles,(2:15),Penelope Ford,NaN,NaN,NaN,NaN,NaN,Layna Lennox,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,AEW Dark: Elevation #28,2021-09-13,Jade Cargill (w/Mark Sterling) defeats Shawna ...,singles,(1:01),Jade Cargill,NaN,NaN,NaN,NaN,NaN,Shawna Reed,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,AEW Dark: Elevation #28,2021-09-13,Nyla Rose (w/Vickie Guerrero) defeats Skye Blu...,singles,(3:33),Nyla Rose,NaN,NaN,NaN,NaN,NaN,Skye Blue,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,AEW Dark: Elevation #28,2021-09-13,Daniel Garcia (w/Jeff Parker & Matt Lee) defea...,singles,(6:13),Daniel Garcia,NaN,NaN,NaN,NaN,NaN,Lee Moriarty,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1285,AEW Dark: Elevation #28,2021-09-13,The Butcher And The Blade (The Blade & The But...,tag,(1:17),The Blade,The Butcher,NaN,NaN,NaN,NaN,Truth Magnum,Turbo Floyd,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
x = Rating(25, 8)
x.mu

Rating()

trueskill.Rating(mu=25.000, sigma=8.333)

In [175]:
def create_player_frame():
    '''
    initializes dataframe
    '''
    player_frame = pd.DataFrame(columns=['name', 'mu', 'sigma'])
    return player_frame

def create_macro_player_frame():
    '''
    initializes by-date dataframe
    '''
    player_frame = pd.DataFrame(columns=['name', 'mu', 'sigma','Date', 'Result'])
    return player_frame


def player_add(name, player_frame):
    '''
    adds player to created player frame
    '''
    x = Rating()
    player_frame  = player_frame.append({'name': name, \
                                         'mu': x.mu, 'sigma': x.sigma}, ignore_index=True)
    
    return player_frame

def player_array(name, player_frame):
    '''
    returns mu, sigma (in that order) of player's trueskill
    '''
    
    mu = player_frame[player_frame['name'] == name]['mu']
    sigma = player_frame[player_frame['name'] == name]['sigma']
    
    mu = list(mu)
    sigma = list(sigma)
    
    mu1 = mu[0]
    sigma1 = sigma[0]
    
    return [mu1, sigma1]

def victory_1v1(winner, loser, player_frame):
    '''
    updates player_frame after a 1v1 matchup that is not a tie
    '''
    
    winner_mu, winner_sigma = player_array(winner, player_frame)
    loser_mu, loser_sigma = player_array(loser, player_frame)
    
    win_rate = Rating(winner_mu, winner_sigma)
    lose_rate = Rating(loser_mu, loser_sigma)
    
    result = rate([(win_rate,), (lose_rate,)], ranks=[0,1])
    (win,), (lose,), = result
    
    player_frame = player_frame[player_frame['name'] != winner]
    player_frame = player_frame[player_frame['name'] != loser]
    
    player_frame  = player_frame.append({'name': loser, \
                                         'mu': lose.mu, 'sigma': lose.sigma}, ignore_index=True)
    player_frame  = player_frame.append({'name': winner, \
                                         'mu': win.mu, 'sigma': win.sigma}, ignore_index=True)
    
    return player_frame

def get_rating(name, player_frame):
    '''
    '''
    mu, sigma = player_array(name, player_frame)
    rating = Rating(mu, sigma)
    return rating

def victory_singles(result_list, player_frame):
    '''
    updates player_frame after any singles matchup that is not a tie (regardless of participant count)
    '''
    rate_pt1 = []
    for item in result_list:
        x = (get_rating(item, player_frame),)
        rate_pt1.append(x)
        
    y = len(result_list)
    rate_pt2 = [0]
    for i in [1] * (y-1):
        rate_pt2.append(1)
        
    result = rate(rate_pt1, ranks=rate_pt2)
    
    player_frame_old = player_frame
    
    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame = player_frame.append({'name': result_list[i], \
                                         'mu': result[i][0].mu, 'sigma': result[i][0].sigma}, ignore_index=True)
        
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[1:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu
    return player_frame

def victory_draw(result_list, player_frame):
    '''
    updates player_frame after any singles matchup that is a tie (regardless of participant count)
    '''
    rate_pt1 = []
    for item in result_list:
        x = (get_rating(item, player_frame),)
        rate_pt1.append(x)
        
    y = len(result_list)
    rate_pt2 = [0]
    for i in [1] * (y-1):
        rate_pt2.append(0)
        
    result = rate(rate_pt1, ranks=[0,0])
    
    player_frame_old = player_frame
    
    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame = player_frame.append({'name': result_list[i], \
                                         'mu': result[i][0].mu, 'sigma': result[i][0].sigma}, ignore_index=True)
        
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    #added 9.27.2021: also just ratings don't move in ties, how about
    for item in result_list[0:]:
        if get_rating(item, player_frame).mu != get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu
   
    return player_frame

def victory_tag(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)

    y1 = Rating(get_rating(result_list[2], player_frame),)
    y2 = Rating(get_rating(result_list[3], player_frame),)

    rate_pt2 = [0, 1]
    
    result = rate([(x1,x2), (y1,y2)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[2:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_threewaytag(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)

    y1 = Rating(get_rating(result_list[2], player_frame),)
    y2 = Rating(get_rating(result_list[3], player_frame),)
    
    z1 = Rating(get_rating(result_list[4], player_frame),)
    z2 = Rating(get_rating(result_list[5], player_frame),)

    rate_pt2 = [0, 1, 1]
    
    result = rate([(x1,x2), (y1,y2), (z1, z2)], ranks=[0,1,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[2:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_fourwaytag(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)

    y1 = Rating(get_rating(result_list[2], player_frame),)
    y2 = Rating(get_rating(result_list[3], player_frame),)
    
    z1 = Rating(get_rating(result_list[4], player_frame),)
    z2 = Rating(get_rating(result_list[5], player_frame),)
    
    v1 = Rating(get_rating(result_list[6], player_frame),)
    v2 = Rating(get_rating(result_list[7], player_frame),)

    rate_pt2 = [0, 1, 1, 1]
    
    result = rate([(x1,x2), (y1,y2), (z1, z2), (v1, v2)], ranks=[0,1,1,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[2:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_tag6(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)
    x3 = Rating(get_rating(result_list[2], player_frame),)

    y1 = Rating(get_rating(result_list[3], player_frame),)
    y2 = Rating(get_rating(result_list[4], player_frame),)
    y3 = Rating(get_rating(result_list[5], player_frame),)

    rate_pt2 = [0, 1]
    
    result = rate([(x1,x2,x3), (y1,y2,y3)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[3:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_tag8(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)
    x3 = Rating(get_rating(result_list[2], player_frame),)
    x4 = Rating(get_rating(result_list[3], player_frame),)

    y1 = Rating(get_rating(result_list[4], player_frame),)
    y2 = Rating(get_rating(result_list[5], player_frame),)
    y3 = Rating(get_rating(result_list[6], player_frame),)
    y4 = Rating(get_rating(result_list[7], player_frame),)

    rate_pt2 = [0, 1]

    
    result = rate([(x1,x2,x3,x4), (y1,y2,y3,y4)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[4:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_tag10(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)
    x3 = Rating(get_rating(result_list[2], player_frame),)
    x4 = Rating(get_rating(result_list[3], player_frame),)
    x5 = Rating(get_rating(result_list[4], player_frame),)

    y1 = Rating(get_rating(result_list[5], player_frame),)
    y2 = Rating(get_rating(result_list[6], player_frame),)
    y3 = Rating(get_rating(result_list[7], player_frame),)
    y4 = Rating(get_rating(result_list[8], player_frame),)
    y5 = Rating(get_rating(result_list[9], player_frame),)

    rate_pt2 = [0, 1]
    
    result = rate([(x1,x2,x3,x4,x5), (y1,y2,y3,y4,y5)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[5:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_tag12(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)
    x3 = Rating(get_rating(result_list[2], player_frame),)
    x4 = Rating(get_rating(result_list[3], player_frame),)
    x5 = Rating(get_rating(result_list[4], player_frame),)
    x6 = Rating(get_rating(result_list[5], player_frame),)

    y1 = Rating(get_rating(result_list[6], player_frame),)
    y2 = Rating(get_rating(result_list[7], player_frame),)
    y3 = Rating(get_rating(result_list[8], player_frame),)
    y4 = Rating(get_rating(result_list[9], player_frame),)
    y5 = Rating(get_rating(result_list[10], player_frame),)
    y6 = Rating(get_rating(result_list[11], player_frame),)

    rate_pt2 = [0, 1]
    
    result = rate([(x1,x2,x3,x4,x5,x6), (y1,y2,y3,y4,y5,y6)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[6:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_handicap3on1(result_list, player_frame, date):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    if date == pd.to_datetime('2021-09-27').normalize():
        
        x1 = Rating(get_rating('Paul Wight', player_frame),)
        x2 = Rating(25,3)
        x3 = Rating(25,3)

        y1 = Rating(get_rating('VSK', player_frame),)
        y2 = Rating(get_rating('CPA', player_frame),)
        y3 = Rating(get_rating('RSP', player_frame),)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2,x3), (y1,y2,y3)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame
    
    if date == pd.to_datetime('2020-08-27').normalize():
        
        x1 = Rating(get_rating('Big Swole', player_frame),)
        x2 = Rating(25,3)
        x3 = Rating(25,3)

        y1 = Rating(get_rating('Dr. Britt Baker DMD', player_frame),)
        y2 = Rating(get_rating('Penelope Ford', player_frame),)
        y3 = Rating(get_rating('Rebel', player_frame),)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2,x3), (y1,y2,y3)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_handicap2on1(result_list, player_frame, date):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    if date == pd.to_datetime('2020-08-12').normalize():
        
        x1 = Rating(get_rating('Lance Archer', player_frame),)
        x2 = Rating(25,3)

        y1 = Rating(get_rating('Jessy Sorensen', player_frame),)
        y2 = Rating(get_rating('Jon Cruz', player_frame),)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2), (y1,y2)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu
                
    if date == pd.to_datetime('2020-07-02').normalize():
        
        x1 = Rating(get_rating('Nyla Rose', player_frame),)
        x2 = Rating(25,1)

        y1 = Rating(get_rating('Kenzie Paige', player_frame),)
        y2 = Rating(get_rating('KiLynn King', player_frame),)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2), (y1,y2)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu
                
    if date == pd.to_datetime('2020-03-04').normalize():
        
        x1 = Rating(get_rating('Chris Jericho', player_frame),)
        x2 = Rating(get_rating('Sammy Guevara', player_frame),)
        
        y1 = Rating(get_rating('Darby Allin', player_frame),)
        y2 = Rating(25,1)
        
        print(x1)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2), (y1,y2)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu
                
    if date == pd.to_datetime('2021-06-18').normalize():
        
        x1 = Rating(get_rating('Ethan Page', player_frame),)
        x2 = Rating(get_rating('Scorpio Sky', player_frame),)
        
        y1 = Rating(get_rating('Darby Allin', player_frame),)
        y2 = Rating(25,1)
        
        print(x1)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2), (y1,y2)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu
                
    if date == pd.to_datetime('2021-09-03').normalize():
        
        x1 = Rating(get_rating('Kris Statlander', player_frame),)
        x2 = Rating(25,3)

        y1 = Rating(get_rating('Rebel', player_frame),)
        y2 = Rating(get_rating('Jamie Hayter', player_frame),)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2), (y1,y2)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu


    return player_frame

### does the deal

For every day since May 1, 2019, checks if there was at least one AEW match that happened that day in the data. If there was, adds each match to the larger results frame. Adds player to the results frame if they are new. If the day is December 31, performs rankings reset.

In [176]:
player_frame = create_player_frame()
macro_player_frame = create_macro_player_frame()

pandas_today = pd.to_datetime('today').normalize()
x = pd.to_datetime('2019-05-01').normalize()

while x < pandas_today:
    
    #optionally can add (all_matches['matchtype'] == 'singles') for specific divisions only
    day_matches = all_matches[(all_matches['Date'] == x)]
    day_results = [item for item in day_matches.result]

    for index, row in day_matches.iterrows():
        placement = [item for item in [row.win1, row.win2, row.win3, row.win4, row.win5, row.win6,\
                                       row.lose1, row.lose2, row.lose3, row.lose4, row.lose5, row.lose6,\
                                      row.lose7, row.lose8] if str(item) != 'nan']
        placement = [alias_collapser(item) for item in placement]

        for item in placement:
            if item not in set(player_frame.name):
                player_frame = player_add(item, player_frame)
        if row.matchtype in ['singles', 'threeway', 'fourway', 'sixway', 'nineway']:
            player_frame = victory_singles(placement, player_frame)
        if row.matchtype in ['draw']:
            player_frame = victory_draw(placement, player_frame)
        if row.matchtype in ['tag']:
            player_frame = victory_tag(placement, player_frame)
        if row.matchtype in ['tag6']:
            player_frame = victory_tag6(placement, player_frame)
        if row.matchtype in ['tag8']:
            player_frame = victory_tag8(placement, player_frame)
        if row.matchtype in ['tag10']:
            player_frame = victory_tag10(placement, player_frame)
        if row.matchtype in ['tag12']:
            player_frame = victory_tag12(placement, player_frame)
        if row.matchtype in ['threewaytag']:
            player_frame = victory_threewaytag(placement, player_frame)
        if row.matchtype in ['fourwaytag']:
            player_frame = victory_fourwaytag(placement, player_frame)
        if row.matchtype in ['handicap3on1']:
            player_frame = victory_handicap3on1(placement, player_frame,x)
        if row.matchtype in ['handicap2on1']:
            player_frame = victory_handicap2on1(placement, player_frame,x)
    
    intermediary = player_frame
    intermediary['Date'] = x
    intermediary['Result'] = ''
    
    for index, row in intermediary.iterrows():
        for item in day_results:
            if str(row['name']) in item:
                intermediary.loc[index, 'Result'] = item
                
    macro_player_frame = pd.concat([macro_player_frame, intermediary])
    
    if (x.month == 12) & (x.day == 31):    
        player_frame = player_frame[~(player_frame.sigma == 8.33)]
        player_frame.mu = [25 + ((i - 25) / 2) for i in player_frame.mu]
        player_frame.sigma = [8.33 for i in player_frame.sigma]
    
    x = x + pd.DateOffset(1)

trueskill.Rating(mu=32.792, sigma=6.425)
trueskill.Rating(mu=31.202, sigma=3.102)


In [177]:
macro_player_frame = macro_player_frame.groupby(['name', 'mu', 'sigma']).aggregate({"Date": "min", "Result": "max"})

In [178]:
x = macro_player_frame.groupby(by=["name"], dropna=False).count()
matchcounts = pd.DataFrame(x['Date'])

player_frame = player_frame.sort_values('name', ascending=True)

In [179]:
player_frame[player_frame.sigma < 8.333].sort_values('mu', ascending=False).to_csv('ranking_output_singles_' + today + '.csv', index=False)


In [180]:
player_frame[player_frame.sigma < 8.333].sort_values('mu', ascending=False).to_csv('ranking_output_' + today + '.csv', index=False)
macro_player_frame.sort_values('mu', ascending=False).to_csv('macro_output_' + today + '.csv')

macro_player_frame.sort_values('mu', ascending=False).to_csv('macro_output.csv')

In [181]:
macro_player_frame.sort_values('Date').filter(like = 'Hayter', axis=0)

Date  \
name         mu        sigma                 
Jamie Hayter 20.858015 6.883007 2019-10-23   
             24.449647 6.315332 2019-11-06   
             24.724824 8.330000 2020-01-01   
             34.679295 5.933275 2021-08-25   
             25.354428 2.960633 2021-09-03   
             24.640759 2.885075 2021-09-08   
             24.392527 2.860903 2021-09-10   

                                                                            Result  
name         mu        sigma                                                        
Jamie Hayter 20.858015 6.883007    Dr. Britt Baker DMD defeats Jamie Hayter (8:40)  
             24.449647 6.315332  Emi Sakura & Jamie Hayter defeat Riho & Shanna...  
             24.724824 8.330000                                                     
             34.679295 5.933275  Jamie Hayter (w/Dr. Britt Baker DMD & Rebel) d...  
             25.354428 2.960633  Kris Statlander defeats Jamie Hayter & Rebel  ...  
             24.640759 2.885075  Ruby Soho defeats Jamie Hayter (w/Dr. Britt Ba...  
             24.392527 2.860903  Kris Statlander, Riho & Ruby Soho defeat Dr. B...

## Double-macro frame (all date comparison)

In [182]:
x = pd.read_csv('macro_output.csv')
x = x.sort_values(['name', 'Date'])

x.Date

4272    2021-09-11
4429    2021-01-19
3891    2021-02-23
4132    2021-03-02
4265    2021-03-09
           ...    
4208    2021-08-19
4551    2021-08-23
3997    2021-08-10
4199    2021-09-11
4430    2021-06-08
Name: Date, Length: 6447, dtype: object

In [183]:
x = pd.read_csv('macro_output.csv')
x = x.sort_values(['name', 'Date'])

double_macro = pd.DataFrame(columns=['name', 'mu', 'sigma', 'Date'])

zz = 0
print(len(set(x.name)))

for guy in set(x.name):
    print(zz)
    print(guy)
    y = x[x.name == guy]
    y = y.reset_index()
    y = y[['name', 'mu', 'sigma', 'Date']]
    y['Date'] = pd.to_datetime(y.Date, format='%Y-%m-%d')
    
    date1 = y.Date[0] #first appearance
    date2 = y.Date[len(y) - 1] #last appearance
    
    cutoff = date2 + relativedelta(months=+3)
    
    print(date1)
    print(cutoff)
    
    check_date = date1
    most_recent_match = date1
    
    while (check_date < cutoff) & (check_date < pandas_today):
        all_dates = list(y.Date)
        
        if (check_date not in all_dates) & (check_date < most_recent_match + pd.DateOffset(90)):
            y = y.append({'name': guy, 'mu': None, 'sigma': None, 'Date': check_date}, ignore_index=True)
            
        if (check_date in all_dates) & (check_date != '2021-01-01 00:00:00'):
            most_recent_match = check_date
            
        check_date = check_date + pd.DateOffset(1)
        
    y = y.sort_values('Date')
    y = y.fillna(method='ffill')
    
    double_macro = pd.concat([double_macro, y])
    
    zz = zz + 1

512
0
Aaron Rourke
2021-05-18 00:00:00
2021-09-28 00:00:00
1
Madi Maxx
2021-08-03 00:00:00
2021-11-03 00:00:00
2
Anthony Bennett
2021-09-20 00:00:00
2021-12-20 00:00:00
3
Bobby Wayward
2020-12-01 00:00:00
2021-04-01 00:00:00
4
Brady Pierce
2020-05-26 00:00:00
2021-04-01 00:00:00
5
Ethan Page
2021-03-07 00:00:00
2021-12-24 00:00:00
6
Marko Stunt
2019-08-31 00:00:00
2021-12-11 00:00:00
7
Colin Delaney
2020-02-04 00:00:00
2021-04-01 00:00:00
8
PAC
2019-08-31 00:00:00
2021-12-10 00:00:00
9
Brandon Bullock
2020-09-04 00:00:00
2021-04-01 00:00:00
10
Alan Angels
2020-04-08 00:00:00
2021-12-20 00:00:00
11
Adrian Jaoude
2021-09-11 00:00:00
2021-12-11 00:00:00
12
Dulce Tormenta
2021-07-19 00:00:00
2021-10-19 00:00:00
13
Viva Van
2021-07-06 00:00:00
2021-10-06 00:00:00
14
Kal Herro
2021-05-25 00:00:00
2021-12-11 00:00:00
15
Dave Dutra
2020-11-03 00:00:00
2021-04-01 00:00:00
16
Terrence Hughes
2020-11-10 00:00:00
2021-10-06 00:00:00
17
Anthony Ogogo
2021-04-14 00:00:00
2021-09-04 00:00:00
18
Steve

149
Trevor Aeon
2021-06-07 00:00:00
2021-09-07 00:00:00
150
MT Nakazawa
2019-06-29 00:00:00
2021-08-05 00:00:00
151
Renee Michelle
2021-02-16 00:00:00
2021-11-09 00:00:00
152
Ryan Pyles
2020-04-07 00:00:00
2021-04-01 00:00:00
153
Maki Ito
2021-02-15 00:00:00
2021-06-15 00:00:00
154
Jimmy Havoc
2019-06-29 00:00:00
2021-04-01 00:00:00
155
Cezar Bononi
2020-09-29 00:00:00
2021-12-11 00:00:00
156
Brandon Gore
2021-09-04 00:00:00
2021-12-11 00:00:00
157
Adrian Alanis
2021-04-26 00:00:00
2021-12-11 00:00:00
158
Mercedes Martinez
2019-11-05 00:00:00
2020-04-01 00:00:00
159
Manny Smith
2021-05-10 00:00:00
2021-08-10 00:00:00
160
KiLynn King
2020-05-26 00:00:00
2021-12-14 00:00:00
161
Brayden Lee
2021-08-24 00:00:00
2021-11-24 00:00:00
162
Daunte Martin
2020-10-27 00:00:00
2021-12-15 00:00:00
163
Serena Deeb
2020-09-02 00:00:00
2021-10-19 00:00:00
164
Falco
2020-12-08 00:00:00
2021-09-14 00:00:00
165
Colt Cabana
2020-03-04 00:00:00
2021-12-20 00:00:00
166
Kenzie Paige
2020-05-06 00:00:00
2021-1

296
Dillon McQueen
2021-05-03 00:00:00
2021-12-11 00:00:00
297
Sage Scott
2021-02-16 00:00:00
2021-11-06 00:00:00
298
Killa Kate
2021-07-27 00:00:00
2021-10-27 00:00:00
299
Ivelisse
2020-07-22 00:00:00
2021-06-02 00:00:00
300
Baron Black
2020-04-29 00:00:00
2021-12-11 00:00:00
301
Big Game Leroy
2020-06-02 00:00:00
2021-04-01 00:00:00
302
Bshp King
2020-09-29 00:00:00
2021-04-01 00:00:00
303
George South Jr.
2020-12-01 00:00:00
2021-04-01 00:00:00
304
MJ Jenkins
2020-07-15 00:00:00
2021-04-01 00:00:00
305
Cash Wheeler
2020-06-10 00:00:00
2021-12-22 00:00:00
306
Trent
2019-05-25 00:00:00
2021-07-21 00:00:00
307
Anna Jay
2020-04-01 00:00:00
2021-12-24 00:00:00
308
Steven Andrews
2020-04-06 00:00:00
2021-08-25 00:00:00
309
Darian Bengston
2021-06-21 00:00:00
2021-12-11 00:00:00
310
Diamante
2020-01-21 00:00:00
2021-12-11 00:00:00
311
Matt Sells
2020-03-24 00:00:00
2021-04-01 00:00:00
312
Dream Girl Ellie
2020-11-10 00:00:00
2021-10-12 00:00:00
313
Randy Summers
2021-03-29 00:00:00
2021-08

443
Adam Page
2019-06-29 00:00:00
2021-10-28 00:00:00
444
Mysterious Movado
2021-09-14 00:00:00
2021-12-14 00:00:00
445
Jorel Nelson
2021-03-15 00:00:00
2021-09-21 00:00:00
446
Marina Tucker
2021-09-11 00:00:00
2021-12-11 00:00:00
447
Rachael Ellering
2020-08-10 00:00:00
2021-04-01 00:00:00
448
JD Drake
2021-02-23 00:00:00
2021-11-25 00:00:00
449
Tesha Price
2020-11-06 00:00:00
2021-11-09 00:00:00
450
Milo Beasley
2021-06-21 00:00:00
2021-09-21 00:00:00
451
Jack Evans
2019-05-25 00:00:00
2021-12-14 00:00:00
452
Sammy Guevara
2019-05-25 00:00:00
2021-11-30 00:00:00
453
Kiah Dream
2021-05-17 00:00:00
2021-08-17 00:00:00
454
Sotheara Chhun
2020-12-08 00:00:00
2021-04-01 00:00:00
455
Matt Hardy
2020-05-06 00:00:00
2021-12-05 00:00:00
456
Eric James
2021-02-16 00:00:00
2021-05-16 00:00:00
457
Doc Gallows
2021-01-13 00:00:00
2021-12-01 00:00:00
458
Khash
2021-09-14 00:00:00
2021-12-14 00:00:00
459
Ashley Vox
2021-01-05 00:00:00
2021-10-06 00:00:00
460
Sean Legacy
2020-09-08 00:00:00
2021-04-

In [184]:
double_macro

,name,mu,sigma,Date
0,Aaron Rourke,20.700536,6.784359,2021-05-18
2,Aaron Rourke,20.700536,6.784359,2021-05-19
3,Aaron Rourke,20.700536,6.784359,2021-05-20
4,Aaron Rourke,20.700536,6.784359,2021-05-21
5,Aaron Rourke,20.700536,6.784359,2021-05-22
...,...,...,...,...
11,Anthony Henry,22.072814,6.992150,2021-09-22
12,Anthony Henry,22.072814,6.992150,2021-09-23
13,Anthony Henry,22.072814,6.992150,2021-09-24
14,Anthony Henry,22.072814,6.992150,2021-09-25


In [185]:
double_macro.sort_values('name', ascending=False).to_csv('double_macro_' + today + '.csv')
double_macro.sort_values('name', ascending=False).to_csv('double_macro.csv')

PermissionError: [Errno 13] Permission denied: 'double_macro.csv'

In [ ]:
double_macro[double_macro.Date == '2021-08-21'].sort_values('mu', ascending=False)

In [ ]:
date1 = pd.to_datetime('2019-05-26').normalize()

while date1 < pandas_today:
    z = double_macro[double_macro.Date == date1]
    z = z.sort_values('mu', ascending=False)
    z = z.reset_index()
    bottom = len(z)
    
    print(date1, '|', z.name[0], '|', z.name[bottom - 1])
    
    date1 = date1 + pd.DateOffset(1)
    


## Upset Hunter

In [ ]:
double_macro

In [ ]:
def get_day_skill(guy, date):
    '''
    Hoo boy.
    
    Goes back one day to get their rating going into the match. If this is their debut, returns 25.
    If it's not their debut but they've been away for a long time, cycles back recursively.
    '''
    if type(guy) == float:
        return np.nan
    truedate = pd.to_datetime(date) - timedelta(days=1)
    filtered_macro = double_macro[double_macro['name'] == alias_collapser(guy)]
    len_filtered = len(filtered_macro)
    try:
        y = list(filtered_macro[filtered_macro.Date == truedate]['mu'])[0]
        return y
    except IndexError:
        try:
            if filtered_macro[filtered_macro.Date == pd.to_datetime(date)].index[0] > 0:
                date_present = False
                while date_present == False:
                    truedate = truedate - timedelta(days=1)
                    date_present = (truedate in list(filtered_macro.Date))
                y = list(filtered_macro[filtered_macro.Date == truedate]['mu'])[0]
                return y
            else:
                return 25
        except IndexError:
            return 25

In [ ]:
get_day_skill(np.nan,'2019-05-01')

In [ ]:
merged_df['Date'] = pd.to_datetime(merged_df['Date'], format='%d.%m.%Y')
merged_df = merged_df.reset_index(drop=True)

In [ ]:
merged_df_head = merged_df.head()

In [ ]:
alias_collapser('Preston Vance')

In [ ]:
cols = ['win1', 'win2', 'win3', 'win4',  'win5', 'win6', \
        'lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'lose6', 'lose7', 'lose8']

for col in cols:
    colname = col + '_skill'
    merged_df[colname] = [get_day_skill(alias_collapser(merged_df[col][i]), merged_df.Date[i]) for i in merged_df.index]

In [ ]:
merged_df

In [ ]:
win_str = merged_df[['win1_skill', 'win2_skill', 'win3_skill', 'win4_skill',  'win5_skill', 'win6_skill']]
win_str['average'] = win_str.mean(axis=1)

merged_df['win_avg'] = win_str.average

lose_str = merged_df[['lose1_skill', 'lose2_skill', 'lose3_skill', 'lose4_skill',\
                      'lose5_skill', 'lose6_skill', 'lose7_skill', 'lose8_skill']]
lose_str['average'] = lose_str.mean(axis=1)

merged_df['lose_avg'] = lose_str.average

merged_df['delta'] = merged_df.win_avg - merged_df.lose_avg

In [ ]:
merged_df.sort_values('delta').to_csv('with_skills.csv')

In [ ]:
merged_df.sort_values('Date', ascending=False).head(10)

In [ ]:
alias_collapser('Lee Johnson')

In [ ]:
tops = merged_df.sort_values('delta')[['Event', 'Date', 'result', 'matchtype', 'win_avg', 'lose_avg', 'delta']]
tops = tops[(tops.win_avg != 25) & (tops.lose_avg != 25)]

In [ ]:
tops.head(10)

In [ ]:
tops.tail(10)

In [ ]:
v = tops.tail(10)

for item in v.result:
    print(item)

In [ ]:
win_loss = pd.DataFrame(columns=['name', 'win', 'loss', 'draw'])
win_loss = win_loss.set_index('name')

cols_win = ['win1', 'win2', 'win3', 'win4',  'win5', 'win6']
cols_lose = ['lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'lose6', 'lose7', 'lose8']

merged_df_alldraws = merged_df[merged_df.matchtype == 'draw']

merged_df_nodraws = merged_df[~(merged_df.matchtype == 'draw')]

for col in cols_win:
    for name in merged_df_nodraws[col].dropna():
        if alias_collapser(name) not in list(win_loss.index):
            win_loss.loc[str(alias_collapser(name))] = [0,0,0]
        win_loss.loc[alias_collapser(name)]['win'] = win_loss.loc[alias_collapser(name)]['win'] + 1
            
for col in cols_lose:
    for name in merged_df_nodraws[col].dropna():
        if alias_collapser(name) not in list(win_loss.index):
            win_loss.loc[str(alias_collapser(name))] = [0,0,0]
        win_loss.loc[alias_collapser(name)]['loss'] = win_loss.loc[alias_collapser(name)]['loss'] + 1
        
for col in ['win1', 'lose1']:
    for name in merged_df_alldraws[col].dropna():
        if alias_collapser(name) not in list(win_loss.index):
            win_loss.loc[str(alias_collapser(name))] = [0,0,0]
        win_loss.loc[alias_collapser(name)]['draw'] = win_loss.loc[alias_collapser(name)]['draw'] + 1

In [ ]:
win_loss.to_csv('win_loss.csv')

In [ ]:
win_loss.sort_values('win', ascending=False)

In [ ]:
win_loss.sort_values('loss', ascending=False)

In [ ]:
win_loss[(win_loss.win == 1) & (win_loss.loss == 0)]

## Historical Upset %

In [ ]:
merged_df[abs(merged_df.delta) > 10].sort_values('delta')

In [ ]:
merged_df['Dynamite'] = merged_df.Event.str.contains('Dynamite', regex=False)
merged_df_2 = merged_df[merged_df['Dynamite'] == True]

In [ ]:
win_probs = pd.DataFrame(columns = ['delta', 'win', 'total', 'wp%'])

i = -0.01
interval = 1
while (i < 40):
    filtered_df = merged_df[(i < abs(merged_df.delta)) & (i + interval > abs(merged_df.delta))]
    win = (filtered_df.delta > 0).sum()
    total = len(filtered_df)
    wp = round(win / total, 2)
    win_probs = win_probs.append({'delta': i + .01, 'win': win, 'total': total, 'wp%': wp}, ignore_index=True)
    i = i + interval
    
win_probs

In [ ]:
win_probs.to_csv('win_probs_' + today + '.csv')

## that one thing

In [ ]:
for_curve = merged_df[['Event', 'Date', 'result', 'matchtype', 'win_avg', 'lose_avg', 'delta']]
for_curve

In [ ]:
for_curve['chalk'] = [1 if i > 0 else 0 for i in for_curve.delta]
for_curve['abs'] = [abs(i) for i in for_curve.delta]

In [ ]:
statframe = for_curve[['chalk', 'Event', 'Date', 'win_avg', 'lose_avg', 'abs', 'matchtype']]

In [ ]:
statframe

In [ ]:
x = pd.get_dummies(statframe, columns=["matchtype"])
x.to_csv("upsets_for_analysis.csv")

In [ ]:
datof = double_macro[double_macro.Date == '2021-09-26'].sort_values('mu', ascending=False)
datof.append({'name': 'Debutee', 'mu': 25, 'sigma': 8.33, 'Date': today_dash}, ignore_index=True).to_csv('datof.csv')

In [ ]:
set(list(statframe['Event']))